# PYTHON: GENERATORS, COROUTINES, NATIVE COROUTINES AND ASYNC/AWAIT

## Generators

In [1]:
def simple_gen():
    yield "Hello"
    yield "World"
 
 
gen = simple_gen()
print(next(gen))
print(next(gen))

Hello
World


In [2]:
def generate_nums():
    num = 0
    while True:
        yield num
        num = num + 1
 
 
nums = generate_nums()
 
for x in nums:
    print(x)
 
    if x > 9:
        break

0
1
2
3
4
5
6
7
8
9
10


**Summary**: 
> A generator function is a function that can pause execution and generate multiple values instead of just returning one value. When called, it gives us a generator object which acts like an iterable. We can use (iterate over) the iterable to get the values one by one.

## Coroutines

In the last section we have seen that using generators we can pull data from a function context (and pause execution). What if we wanted to push some data too? That’s where coroutines comes into play. The *yield* keyword we use to pull values can also be used as an expression (on the right side of “=”) inside the function. We can use the *send()* method on a generator object to pass values back into the function. This is called “generator based coroutines”. Here’s an example:

In [4]:
def coro():
    hello = yield "Hello"
    yield hello
 
 
c = coro()
print(next(c))
print(c.send("World"))

Hello
World


When we’re using generator based coroutines, by the terms “generator” and “coroutine” we usually mean the same thing. Though they are not exactly the same thing, it is very often used interchangeably in such cases. However, with Python 3.5 we have *async/await* keywords along with native coroutines. We will discuss those later in this post.

## Async I/O and the asyncio module

In [5]:
import asyncio
import datetime
import random
 
 
@asyncio.coroutine
def display_date(num, loop):
    end_time = loop.time() + 50.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(random.randint(0, 5))
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()

RuntimeError: This event loop is already running

Loop: 1 Time: 2019-01-11 09:16:50.645548
Loop: 2 Time: 2019-01-11 09:16:50.645619
Loop: 1 Time: 2019-01-11 09:16:50.645885
Loop: 1 Time: 2019-01-11 09:16:50.646044
Loop: 1 Time: 2019-01-11 09:16:50.646091
Loop: 1 Time: 2019-01-11 09:16:51.647149
Loop: 1 Time: 2019-01-11 09:16:53.648446
Loop: 2 Time: 2019-01-11 09:16:54.648037
Loop: 1 Time: 2019-01-11 09:16:54.649357
Loop: 1 Time: 2019-01-11 09:16:56.650906
Loop: 1 Time: 2019-01-11 09:16:56.651007
Loop: 2 Time: 2019-01-11 09:16:57.650182
Loop: 1 Time: 2019-01-11 09:17:00.654737
Loop: 1 Time: 2019-01-11 09:17:00.654837
Loop: 2 Time: 2019-01-11 09:17:01.652625
Loop: 2 Time: 2019-01-11 09:17:01.652726
Loop: 2 Time: 2019-01-11 09:17:03.655207
Loop: 1 Time: 2019-01-11 09:17:04.656619
Loop: 2 Time: 2019-01-11 09:17:08.660042
Loop: 1 Time: 2019-01-11 09:17:08.660136
Loop: 1 Time: 2019-01-11 09:17:09.661589
Loop: 2 Time: 2019-01-11 09:17:12.661731
Loop: 1 Time: 2019-01-11 09:17:14.663952
Loop: 2 Time: 2019-01-11 09:17:16.664391
Loop: 1 Time: 20

The code is pretty self explanatory. We create a coroutine display_date(num, loop) which takes an identifier (number) and an event loop and continues to print the current time. Then it used the yield from keyword to await results from asyncio.sleep() function call. The function is a coroutine which completes after a given seconds. So we pass random seconds to it. Then we use asyncio.ensure_future() to schedule the execution of the coroutine in the default event loop. Then we ask the loop to keep running.

If we see the output, we shall see that the two coroutines are executed concurrently. When we use yield from, the event loop knows that it’s going to be busy for a while so it pauses execution of the coroutine and runs another. Thus two coroutines run concurrently (but not in parallel since the event loop is single threaded).

Just so you know, **yield from** is a nice syntactic sugar for 

> `for x in asyncio.sleep(random.randint(0, 5)): yield x` 

making async codes cleaner.

## Native Coroutines and async/await

In [1]:
import asyncio
import datetime
import random
 
 
async def display_date(num, loop, ):
    end_time = loop.time() + 50.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(random.randint(0, 5))
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()

RuntimeError: This event loop is already running

Loop: 1 Time: 2019-01-11 09:23:48.946667
Loop: 2 Time: 2019-01-11 09:23:48.946770
Loop: 1 Time: 2019-01-11 09:23:48.946960
Loop: 2 Time: 2019-01-11 09:23:51.950971


## Native vs Generator Based Coroutines: Interoperability

Despite the differences, we can interoperate between those. We just need to add *\@types.coroutine* decorator to old generator based ones. Then we can use one from inside the other type. That is we can *await* from generator based coroutines inside a native coroutine and *yield from* native coroutines when we are inside generator based coroutines. Here’s an example:

In [2]:
import asyncio
import datetime
import random
import types
 
 
@types.coroutine
def my_sleep_func():
    yield from asyncio.sleep(random.randint(0, 5))
 
 
async def display_date(num, loop, ):
    end_time = loop.time() + 50.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        await my_sleep_func()
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()

RuntimeError: This event loop is already running

Loop: 1 Time: 2019-01-11 09:27:51.003728
Loop: 2 Time: 2019-01-11 09:27:51.003796
Loop: 1 Time: 2019-01-11 09:27:55.005485
Loop: 2 Time: 2019-01-11 09:27:55.006280
Loop: 2 Time: 2019-01-11 09:27:58.009771
Loop: 1 Time: 2019-01-11 09:27:59.008123
Loop: 2 Time: 2019-01-11 09:27:59.010537


# Event Loop

On any platform, when we want to do something asynchronously, it usually involves an event loop. An event loop is a loop that can register tasks to be executed, execute them, delay or even cancel them and handle different events related to these operations. Generally, we schedule multiple async functions to the event loop. The loop runs one function, while that function waits for IO, it pauses it and runs another. When the first function completes IO, it is resumed. Thus two or more functions can co-operatively run together. This the main goal of an event loop.

## Fitting Event Loop and Future/Task Together

It’s simple. We need an event loop and we need to register our future/task objects with the event loop. The loop will schedule and run them. We can add callbacks to our future/task objects so that we can be notified when a future has it’s results.

Very often we choose to use coroutines for our work. We wrap a coroutine in Future and get a Task object. When a coroutine yields, it is paused. When it has a value, it is resumed. When it returns, the Task has completed and gets a value. Any associated callback is run. If the coroutine raises an exception, the Task fails and not resolved.

In [3]:
import asyncio
 
 
@asyncio.coroutine
def slow_operation():
    # yield from suspends execution until
    # there's some result from asyncio.sleep
 
    yield from asyncio.sleep(1)
 
    # our task is done, here's the result
    return 'Future is done!'
 
 
def got_result(future):
    print(future.result())
 
 
# Our main event loop
loop = asyncio.get_event_loop()
 
# We create a task from a coroutine
task = loop.create_task(slow_operation())
 
# Please notify us when the task is complete
task.add_done_callback(got_result)
 
# The loop will close when the task has resolved
loop.run_until_complete(task)

RuntimeError: This event loop is already running

Future is done!


In [4]:
import asyncio
 
 
async def slow_operation():
    await asyncio.sleep(1)
    return 'Future is done!'
 
 
def got_result(future):
    print(future.result())
 
    # We have result, so let's stop
    loop.stop()
 
 
loop = asyncio.get_event_loop()
task = loop.create_task(slow_operation())
task.add_done_callback(got_result)
 
# We run forever
loop.run_forever()

RuntimeError: This event loop is already running

Future is done!


# Callback Style Async

While many asynchronous libraries exist in Python, the most popular ones are probably Tornado and gevent. As we have already talked about gevent, lets focus a little on how Tornado works. Tornado is an asynchronous web framework that uses the callback style to do asynchronous network I/O. A callback is a function, and it means “Once this is done, execute this function”. It’s basically a “when done” hook for your code. In other words a callback is like when you call a customer service line, and immediately leave your number and hang up, so they can call you back when they are available, rather than having to wait on hold forever.

In [1]:
import tornado.ioloop
from tornado.httpclient import AsyncHTTPClient
urls = ['http://www.google.com', 'http://www.yandex.ru', 'http://www.python.org']

def handle_response(response):
    if response.error:
        print("Error:", response.error)
    else:
        url = response.request.url
        data = response.body
        print('{}: {} bytes: {}'.format(url, len(data), data))

http_client = AsyncHTTPClient()
for url in urls:
    http_client.fetch(url, handle_response)
    
tornado.ioloop.IOLoop.instance().start()

RuntimeError: This event loop is already running

http://www.google.com: 49439 bytes: b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content="&#1055;&#1086;&#1080;&#1089;&#1082; &#1080;&#1085;&#1092;&#1086;&#1088;&#1084;&#1072;&#1094;&#1080;&#1080; &#1074; &#1080;&#1085;&#1090;&#1077;&#1088;&#1085;&#1077;&#1090;&#1077;: &#1074;&#1077;&#1073; &#1089;&#1090;&#1088;&#1072;&#1085;&#1080;&#1094;&#1099;, &#1082;&#1072;&#1088;&#1090;&#1080;&#1085;&#1082;&#1080;, &#1074;&#1080;&#1076;&#1077;&#1086; &#1080; &#1084;&#1085;&#1086;&#1075;&#1086;&#1077; &#1076;&#1088;&#1091;&#1075;&#1086;&#1077;." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="uuQdWZJi2lWatydd6lQKJw==">(function(){window.google={kEI:\'Zz44XKHgCM3zsAGMx6jICw\',kEXPI:\'0,1353747,57,1957,1017,624,782,698,527,731,325,653,820,30,195,1033

http://www.yandex.ru: 132662 bytes: b'<!DOCTYPE html><html class="i-ua_js_no i-ua_css_standart i-ua_browser_ i-ua_browser_desktop i-ua_platform_other" lang="ru"><head xmlns:og="http://ogp.me/ns#"><meta http-equiv=Content-Type content="text/html;charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title>\xd0\xaf\xd0\xbd\xd0\xb4\xd0\xb5\xd0\xba\xd1\x81</title><link rel="shortcut icon" href="//yastatic.net/iconostasis/_/8lFaTHLDzmsEZz-5XaQg9iTWZGE.png"><link rel="apple-touch-icon" href="//yastatic.net/iconostasis/_/5mdPq4V7ghRgzBvMkCaTzd2fjYg.png" sizes="76x76"><link rel="apple-touch-icon" href="//yastatic.net/iconostasis/_/s-hGoCQMUosTziuARBks08IUxmc.png" sizes="120x120"><link rel="apple-touch-icon" href="//yastatic.net/iconostasis/_/KnU823iWwj_vrPra7x9aQ-4yjRw.png" sizes="152x152"><link rel="apple-touch-icon" href="//yastatic.net/iconostasis/_/wT9gfGZZ80sP0VsoR6dgDyXJf2Y.png" sizes="180x180"><link rel="alternate" type="application/rss+xml" title="\xd0\x9d\xd0\xbe\xd0\xb2

## Green Threads

Green threads are a primitive level of asynchronous programming. A green thread looks and feels exactly like a normal thread, except that the threads are scheduled by application code rather than by hardware. Gevent is a well known python library for using green threads. Gevent is basically green threads + eventlet, a non-blocking I/O networking library. Gevent monkey patches common python libraries to have non-blocking I/O. Here is an example using gevents to make requests to multiple urls at once:


In [2]:
import gevent.monkey
from urllib.request import urlopen
gevent.monkey.patch_all()
urls = ['http://www.google.com', 'http://www.yandex.ru', 'http://www.python.org']

def print_head(url):
    print('Starting {}'.format(url))
    data = urlopen(url).read()
    print('{}: {} bytes: {}'.format(url, len(data), data))

jobs = [gevent.spawn(print_head, _url) for _url in urls]

gevent.wait(jobs)

ModuleNotFoundError: No module named 'gevent'

## Comparisons
**Green Threads Style**
* Threads are controlled at the application level, rather than hardware
* Feel like threads; Good for those who understand threading
* Includes all the problems of normal thread-based programming other than CPU context switching

**Callback Style**
* Not like threaded programs at all
* Threads/coroutines are invisible to the programmer
* Callbacks swallow exceptions
* Callbacks are not gather-able
* Callback after callback gets confusing and hard to debug